In [21]:
import chess
import chess.engine
import math
from typing import List
import torch
import time
import random
from numpy import float64, int64, uint64


In [22]:
from pathlib import Path
import sys
sys.path.append(str(Path.cwd() / ".." / "src"))
from mcts import Node, Edge, MCTS
from model_wrapper import ModelWrapper
from leela_cnn import LeelaCNN, board_to_tensor
import matplotlib.pyplot as plt
from matplotlib import animation
import matplotlib.patches as patches

In [23]:
BOT_TIME_1 = 0.01
BOT_TIME_2 = 0.1

In [24]:
# init logic
model_name = "checkpoint_2400_20251116_051414.pth"
model_path = "./models/" + model_name


model = LeelaCNN(10, 128)
model.load_state_dict(torch.load(model_path, weights_only=True, map_location=torch.device('mps')))

# wrap the model
wrapped_model = ModelWrapper(model)

# create mcts with the model 
mcts1 = MCTS(wrapped_model, c_puct=float64(30.0))


In [25]:
# init logic
model_name = "checkpoint.pth"
model_path = "./models/" + model_name

model = LeelaCNN(10, 128)
model.load_state_dict(torch.load(model_path, weights_only=True, map_location=torch.device('mps')))

# wrap the model
wrapped_model = ModelWrapper(model)

# create mcts with the model 
mcts2 = MCTS(wrapped_model)

In [26]:
import chess

def is_near_endgame(board: chess.Board) -> bool:
    """
    Very rough 'is this approximately an endgame?' heuristic.

    Idea:
      - Assign simple material values.
      - Sum *all* material except kings for both sides.
      - If there's not much material left, call it (near) endgame.
    """
    values = {
        chess.PAWN:   1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK:   5,
        chess.QUEEN:  9,
    }

    total_material = 0
    for piece in board.piece_map().values():
        if piece.piece_type == chess.KING:
            continue
        total_material += values[piece.piece_type]

    # Threshold is the only magic number here.
    # Full game starts ~78 points (both sides).
    # 14 means < ~20% of total material left.
    return total_material <= 30


In [27]:
def my_bot_move_1(board: chess.Board) -> chess.Move:
    ponder_time = int(BOT_TIME_1 * 1e9)
    move = mcts1.ponder_time(board=board, ponder_time_ns=ponder_time)
    # check if endgame, then move randomly
    # if is_near_endgame(board):
    #     return random.choice(list(board.legal_moves))
    return move


In [28]:
def my_bot_move_2(board: chess.Board) -> chess.Move:
    # ponder_time = int(BOT_TIME_2 * 1e9)
    # move = mcts2.ponder_time(board=board, time_ns=ponder_time)
    return random.choice(list(board.legal_moves))
    return move

In [29]:
def play_bot_1_vs_bot_2_game() -> float:
    """
    Plays a game between bot 1 and bot 2.
    Returns 1.0 if bot 1 wins, 0.5 for draw, 0.0 for loss.
    """
    board = chess.Board()
    my_is_white = True  # bot 1 plays white

    while not board.is_game_over():
        if board.turn == chess.WHITE:
            move = my_bot_move_1(board)
        else:
            move = my_bot_move_2(board)

        board.push(move)
        # display(board._repr_svg_(), clear=True)


    result = board.result()
    if result == "1-0":
        return 1.0 if my_is_white else 0.0
    elif result == "0-1":
        return 0.0 if my_is_white else 1.0
    else:
        return 0.5
def play_games_bot_1_vs_bot_2(num_games: int) -> float:
    bot_1_wins = 0
    bot_2_wins = 0
    ties = 0

    for _ in range(num_games):
        print("Starting game", _ + 1)
        result = play_bot_1_vs_bot_2_game()
        if result == 1.0:
            bot_1_wins += 1
        elif result == 0.0:
            bot_2_wins += 1
        else:
            ties += 1
        print("Game", _ + 1, "result:", result)

    print(f"Bot 1 wins: {bot_1_wins}, Bot 2 wins: {bot_2_wins}, Ties: {ties}")
    return bot_1_wins / num_games


In [30]:
# play the games!
num_games = 10
play_games_bot_1_vs_bot_2(num_games)


Starting game 1
Game 1 result: 0.0
Starting game 2
Game 2 result: 0.0
Starting game 3
Game 3 result: 0.0
Starting game 4
Game 4 result: 0.5
Starting game 5
Game 5 result: 0.5
Starting game 6
Game 6 result: 0.5
Starting game 7
Game 7 result: 0.0
Starting game 8
Game 8 result: 0.0
Starting game 9
Game 9 result: 0.0
Starting game 10
Game 10 result: 0.5
Bot 1 wins: 0, Bot 2 wins: 6, Ties: 4


0.0